In [1]:
import spotipy
import pandas as pd
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials
import json
from pandas.io.json import json_normalize
import glob


#pd.set_option('display.height', 1000)
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

#personal developer key from dev.spotify.com
cid ="cf4404fee2774e0da9efde8b60101fac" 
secret = "77b298babf78448ca28364684ac892df"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [2]:
StreamingFiles = glob.iglob('MyData/StreamingHistory[0-9].json')
data = pd.DataFrame()

for file in StreamingFiles:
    with open(file) as f:
        #print(file)
        stream_history = json.load(f)
        table = json_normalize(stream_history)
        #print(len(table))
        data = data.append(table)

In [3]:
#Sort by number of track plays per artist
artist_counts = data.artistName.value_counts()
data = data.join(artist_counts, on=['artistName'], how='left', lsuffix='', rsuffix='_counts', sort=True)
data = data.sort_values(by=['artistName_counts'], ascending=False)

In [4]:
genre_list = [];
for ii in range(0, len(artist_counts)):
    artist_name = artist_counts.index[ii]
    #display(artist_name)
    searchResult = sp.search(artist_name, limit=1, type='artist')
    if (len(searchResult['artists']['items']) > 0): 
        artist = searchResult['artists']['items'][0]
    else:
        genre_list.append('null')
        pass
    #display(artist)
    if len(artist['genres']) > 0:   
        #display(artist['genres'][0])
        genre_list.append(artist['genres'])
        #data_subset.iloc[ii,5] = artist['genres'][0]
    else:
        genre_list.append('null')
        pass
        #data_subset.iloc[ii]['genres'] = 'none'

ConnectionError: HTTPSConnectionPool(host='api.spotify.com', port=443): Max retries exceeded with url: /v1/search?q=Charli+XCX&limit=1&offset=0&type=artist (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11c9a4550>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [5]:
series_index = artist_counts.index[0:len(genre_list)]

artists_series = pd.Series(genre_list, index=series_index)

In [6]:
display(artists_series)

Anderson .Paak                                   [escape room, hip hop, rap]
Kevin Abstract                                [hip hop, lgbtq+ hip hop, rap]
Sleepless                                                               null
Dreamville                                                    [hip hop, rap]
JID                                      [hip hop, rap, underground hip hop]
                                                 ...                        
Thin Lizzy                 [ambient, art pop, drone, electra, experimenta...
Fountains Of Wayne         [brighton indie, garage rock, indie pop, moder...
Paul "Bunyan" Rosenberg    [dance pop, hyphy, pop rap, rap, southern hip ...
Trudy and the Romance      [australian electropop, australian indie, down...
Father John Misty          [adult standards, dixieland, jazz trumpet, new...
Length: 1051, dtype: object

In [7]:
a

1040